# Atrapar porcentajes quiniela

In [1]:
import bs4 as bs
import urllib
import urllib.request
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import time
import string
import pprint

import io
from urllib.request import urlopen, Request
import requests

import http.client
from http.client import IncompleteRead

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

print (webdriver.__version__)

4.15.2


In [12]:
URL = "https://www.eduardolosilla.es/quiniela/ayudas/porcentajes"

options = webdriver.ChromeOptions()
service = ChromeService(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

print(driver)

driver.get(URL)

firstTime = True

jornada = -1
# for every "jornada"
while jornada < 58:
    jornada += 1
    
    # Click on the "jornada" tab
    select = Select(driver.find_element("id",'u-hidden-scrollbar'))
    select.select_by_value(str(jornada))
    
    time.sleep(randint(5,10))  
    
    cajas = driver.find_elements("xpath","//div[@class='c-caja_base m-no-fixed-size']")
    for caja in cajas:
        cajaTable = caja.text.split('\n')
        #print(caja.text)
        #print(cajaTable)

        ii = 0
        while(ii < (5*14)):
            
            line_ = cajaTable[ii]
            #print(line_)
            saux = (cajaTable[ii+1]).split(' - ')
            localTeamName_ = saux[0]
            awayTeamName_ = saux[1]
            print(localTeamName_)
            print(awayTeamName_)
            
            df = pd.DataFrame(columns=['Season', 'Jornada', 'Line', 'awayTeamName', 'localTeamName'])
            
            df["Season"] = "2021/22"
            df["Jornada"] = jornada
            df["Line"] = line_
            df["awayTeamName"] = awayTeamName_
            df["localTeamName"] = localTeamName_
            
            print(df)
            
            if not firstTime:
                df_ant = pd.concat([df, df_ant], ignore_index=True)
            else: 
                df_ant = df
                firstTime = False
            
            ii += 5
            #print(df_ant)
    #print(df_ant)

<selenium.webdriver.chrome.webdriver.WebDriver (session="43d6dcb6b95649ea81b86d77ce9fe837")>
R.SOCIEDAD
GIRONA
Empty DataFrame
Columns: [Season, Jornada, Line, awayTeamName, localTeamName]
Index: []
LAS PALMAS
MALLORCA
Empty DataFrame
Columns: [Season, Jornada, Line, awayTeamName, localTeamName]
Index: []
ATH.CLUB
R.MADRID
Empty DataFrame
Columns: [Season, Jornada, Line, awayTeamName, localTeamName]
Index: []
CELTA
OSASUNA
Empty DataFrame
Columns: [Season, Jornada, Line, awayTeamName, localTeamName]
Index: []
VILLARREAL
BETIS
Empty DataFrame
Columns: [Season, Jornada, Line, awayTeamName, localTeamName]
Index: []
GETAFE
BARCELONA
Empty DataFrame
Columns: [Season, Jornada, Line, awayTeamName, localTeamName]
Index: []
CÁDIZ
ALAVÉS
Empty DataFrame
Columns: [Season, Jornada, Line, awayTeamName, localTeamName]
Index: []
RACING S.
EIBAR
Empty DataFrame
Columns: [Season, Jornada, Line, awayTeamName, localTeamName]
Index: []
ELCHE
R.FERROL
Empty DataFrame
Columns: [Season, Jornada, Line, awayTe

KeyboardInterrupt: 

In [9]:
df

,Season,Jornada,Line,awayTeamName,localTeamName
